# Task 1. Initialize Vertex AI in a Colab Enterprise notebook

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 54.2 MB/s eta 0:00:00


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-03-f47a071679ee"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Task 2. Load a generative model

In [4]:
model = GenerativeModel("gemini-pro")

# Task 3. Define the output format & specify constraints

In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [6]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```
{
  "turns": [
    {
      "speaker": "Customer",
      "utterance": "Hi, can I get a cheeseburger and large fries, please?"
    },
    {
      "speaker": "Restaurant employee",
      "utterance": "Coming right up! Anything else you'd like to add to your order?"
    },
    {
      "speaker": "Customer",
      "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
    },
    {
      "speaker": "Restaurant employee",
      "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
    }
  ]
}
```


In [7]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food": [
    {
      "item": "cheeseburger",
      "quantity": 1
    }
  ],
  "drinks": [
    {
      "item": "orange juice",
      "size": "small",
      "quantity": 1
    }
  ],
  "other": [
    {
      "item": "fries",
      "size": "large",
      "quantity": 1,
      "modifier": "ketchup on the side"
    }
  ]
}
```



# Task 4. Assign a persona or role

In [8]:
chat = model.start_chat()

In [9]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Monstera Deliciosa Care Guide: 

**Light:** Your Monstera will thrive in bright, indirect light. Avoid direct sun, which can scorch its leaves. East-facing windows are ideal. 

**Watering:** Allow the top inch of soil to dry out before watering. Overwatering is a common killer of Monsteras. Water deeply, ensuring excess drains through the pot. During winter, water less frequently. 

**Humidity:** Aim for a humidity level between 60-80%. Grouping plants, using a pebble tray, or a humidifier can increase humidity. Misting the leaves occasionally can also help. 

**Fertilizer:** Feed your Monstera with a balanced, diluted liquid fertilizer monthly during spring and summer. Avoid fertilizing during fall and winter. 

**Support:** As Monsteras grow taller, they require support. Provide a moss pole or wooden stake for your plant to climb, encouraging healthy growth. 

**Pruning:** Pruning helps maintain your Monstera's size and shape and encourages branching. Using sharp, sterilized shear

In [10]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

Hey there! It's me, your Monstera deliciosa, also known as the Swiss cheese plant! I appreciate you taking care of me, and I want to help you understand what I need to thrive in your loving home. 

Here's a quick peek into the life of a happy Monstera:

**Light:** I'm a sun-loving gal, but I'm not a fan of harsh, direct sunlight. Think of me like Goldilocks; I prefer **bright, indirect light**. A spot near a window that gets filtered sunshine would be perfect!

**Watering:** Don't let my large, lush leaves fool you! I prefer my soil to be slightly dry between waterings. Think "damp" rather than "soggy." In the summer, I might need a drink every week or so, but during winter, I can wait a bit longer. Feel free to stick your finger in the soil to check the moisture level before giving me a drink.

**Humidity:** My tropical roots crave humidity. If the air feels a bit dry, especially during winter, consider using a humidifier or grouping me with other plants to increase the humidity aroun

# Task 5. Include examples

In [11]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

## Customer Message Analysis:

**Message:** Our department needs a custom gen AI solution. We have a budget to explore our idea. Do you have capacity to get started on something soon?

**Likelihood:** 3

**Explanation:** 

This message indicates a high likelihood that the customer is ready to move forward with their project. They have already:

* Identified a need for a custom AI solution.
* Allocated a budget for exploration and development. 
* Expressed urgency by inquiring about the availability to start soon.

These factors suggest that the customer is serious about their project and is likely to hire a developer within the next month, especially if the consulting service can demonstrate the capacity to take on the project promptly. 



# Task 6. Experiment with parameter values

In [12]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class! 



In [13]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog get kicked out of the band? \
\
Because he was always croaking the same tune.


# Task 7. Utilize fallback responses

In [14]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery! 



In [15]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramics are made from clay, whereas porcelain requires the addition of other materials, such as feldspar and quartz. This makes porcelain stronger, whiter, and more heat-resistant than ceramics.


# Task 8. Add contextual information

In [16]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

The location of items in a grocery store can vary depending on the specific store layout. However, here's a general guide to help you find the items you listed:

* **Paper plates:** Look for paper plates in the **paper goods aisle**, which is typically located near the **disposable cups and napkins**. 
* **Mustard:** Mustard is usually found in the **condiment aisle**, which is often located near the **pickles and relishes**. You may also find it in the **international aisle** if you're looking for specific types of mustard, such as Dijon or Chinese mustard.
* **Potatoes:** Potatoes are usually found in the **produce section**, often near other root vegetables like carrots and onions. They may also be located in a **separate bin** near the **fresh fruit** section.

If you're having trouble finding something, don't hesitate to ask a store employee for assistance. 



In [17]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Based on Michael's Grocery Store aisle layout, here's where you can find the items you're looking for:

* **Paper plates:** Aisle 17 (Household & Cleaning Supplies)
* **Mustard:** Aisle 8 (Condiments & Spices)
* **Potatoes:** Aisle 2 (Vegetables) 



# Task 9. Structure prompts with prefixes or tags

In [18]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Allison, I think I might have found the perfect match for you: Felix! 

He shares your love of classical music and is an avid Beethoven enthusiast.  Imagine discussing your favorite concertos over an amazing German dinner, like homemade spaetzle. He's also into swimming, so you two could enjoy fun beach trips or laps at the pool together. 

Felix even seems open-minded about trying new cuisines, like your beloved ramen. He might be the perfect partner to explore your culinary adventures!  



# Task 10. Use system instructions

In [19]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Oh my, there's a whole universe of incredible musicians worth studying!  It all depends on what tickles your eardrums! Want to explore the mathematical genius of Baroque counterpoint?  Dive into Bach!  Feeling the urge to unpack complex harmonies and orchestration?  Wagner's your guy!  

Looking for innovators?  Miles Davis, constantly pushing the boundaries of jazz, or perhaps Beethoven, who took the symphony to dramatic new heights? 

And don't forget the queens!  Billie Holiday, with her heartbreaking vocals, or Sister Rosetta Tharpe, the godmother of rock and roll!  

Tell me, what kind of music sets your soul on fire?  Then, I can point you to some true masters! 🎶🧐🎤🎸🎺🎷🎹🎻🎼🎤🎧🎶


# Task 11. Demonstrate Chain-of-Thought

In [20]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

## Today's Production:

* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

## Tomorrow's Production:

* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory (reconfigured) + 1 factory * 30 units/day/factory (half output) = 90 units/day
* Low efficiency factories: 1 factory * 15 units/day/factory (half output) = 15 units/day
* **Total production tomorrow: 300 units/day + 90 units/day + 15 units/day = 405 units/day** 



# Task 12. Break down complex tasks

In [21]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

## 5 Metaphors Comparing TPUs and GPUs:

1. **Factory vs. Workshop:** 
    - **TPU:** A massive factory with specialized machines for specific tasks, churning out results efficiently but lacking flexibility.
    - **GPU:** A skilled craftsperson's workshop with versatile tools, offering adaptability for diverse projects but potentially slower for repetitive tasks.

2. **Assembly Line vs. Customization:**
    - **TPU:** A highly optimized assembly line, excelling at producing identical items in large quantities with incredible speed.
    - **GPU:** A custom fabrication studio, adept at crafting unique products with intricate details but requiring more time and manual adjustments.

3. **Symphony Orchestra vs. Jazz Band:**
    - **TPU:** A meticulously coordinated symphony orchestra, playing with perfect precision and power but lacking individual improvisation.
    - **GPU:** A dynamic jazz band, capable of spontaneous, creative riffs and variations during performance but requiring closer

In [22]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## Metaphor 3: Symphony Orchestra vs. Jazz Band 

This metaphor resonates most strongly with my current understanding of TPUs and GPUs.  

**The Symphony Orchestra (TPU):**

* **Meticulous coordination:** Like a symphony orchestra, TPUs excel at tasks requiring precise coordination between many individual processing units. This makes them ideal for highly specialized tasks, especially those involving massive datasets and repetitive operations, such as machine learning training. 

* **Power and precision:** Similar to the powerful sound and nuanced control achieved by an orchestra, TPUs deliver high performance and efficiency for their intended purpose. 

* **Limited improvisation:** However, just as a symphony orchestra typically follows a pre-defined score with limited room for improvisation, TPUs are less flexible when dealing with tasks requiring creativity or adapting to changing conditions.

**The Jazz Band (GPU):**

* **Spontaneity and flexibility:** Like a dynamic jazz band, GPU

In [23]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## Demystifying TPUs and GPUs: A Symphony vs. Jazz Band Analogy

Ever wondered what differentiates TPUs from GPUs? While both are powerhouses in the world of computing, their strengths lie in distinct domains. To grasp this difference, let's delve into a musical analogy—the **Symphony Orchestra vs. the Jazz Band**.

Imagine a **Symphony Orchestra**: a meticulously coordinated ensemble where each instrument plays its part to perfection. This precision mirrors the power of TPUs, excelling at tasks demanding meticulous collaboration between processing units. Think massive datasets, repetitive operations, and machine learning training – the orchestra's forte.

But what about flexibility and improvisation? Here's where the **Jazz Band** takes the stage. Like a dynamic ensemble, GPUs adapt to diverse workloads and unexpected situations. They're masters of real-time graphics, interactive applications, and tasks requiring on-the-fly thinking.

While the orchestra's strength lies in its powerfu

# Task 13. Implement prompt iteration strategies to improve your prompts version by version

Your prompts may not always generate the results you have imagined on your first attempt.

A few steps you can take to iterate on your prompts include:

Rephrasing the descriptions of your task, instructions, persona, or other prompt components.
Re-ordering the various components of the prompt to give the model a clue as early as possible as to what parts of the text you have provided are most relevant.
Breaking your task up into multiple, smaller tasks.